In [ ]:
%pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3, MobileNetV3Small
from tensorflow.keras import layers, models
from keras_tuner import HyperModel
from kerastuner.tuners import RandomSearch



# Define the hypermodel
class WasteClassificationHyperModel(HyperModel):
    def __init__(self, dataset_directory, num_classes):
        self.dataset_directory = dataset_directory
        self.num_classes = num_classes
    
    def build(self, hp):
        # Choose the model
        model_type = hp.Choice('model_type', ['InceptionV3', 'MobileNetV3', 'DenseNet201'])
        if model_type == 'InceptionV3':
            base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        elif model_type == 'DenseNet201':
            base_model = tf.keras.applications.DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        else:
            base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        
        # Freeze the base model
        base_model.trainable = False
        
        # Define the model
        model = models.Sequential([
            base_model,
            layers.MaxPooling2D(),
            # layers.Dense(hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
            layers.Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1)),
            # layers.Flatten(),
            layers.Dense(self.num_classes, activation='softmax')
        ])
        
        accuracy = tf.keras.metrics.Accuracy
        # loss = tf.keras.metrics.Loss 
        precision = tf.keras.metrics.Precision 
        recall = tf.keras.metrics.Recall 
        # Compile the model
        model.compile(
            optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
            loss='categorical_crossentropy',
            metrics=['accuracy', 'precision', 'recall']
        )
        
        return model


2024-08-28 21:39:30.187408: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-28 21:39:30.364928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 21:39:30.425227: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 21:39:30.442610: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 21:39:30.574760: I tensorflow/core/platform/cpu_feature_guar

In [3]:

# Define the data generator with augmentation
def create_data_generator(hp, dataset_directory):
    augmentation_technique = hp.Choice('augmentation_technique', ['none', 'flip', 'rotation', 'zoom'])
    
    if augmentation_technique == 'flip':
        datagen = ImageDataGenerator(horizontal_flip=True)
    elif augmentation_technique == 'rotation':
        datagen = ImageDataGenerator(rotation_range=20)
    elif augmentation_technique == 'zoom':
        datagen = ImageDataGenerator(zoom_range=0.2)
    else:
        datagen = ImageDataGenerator()
    
    generator = datagen.flow_from_directory(
        dataset_directory,
        target_size=(224, 224),
        batch_size=hp.Int('batch_size', min_value=16, max_value=64, step=16),
        class_mode='categorical'
    )
    
    return generator


In [4]:
import sys
from tensorflow.keras.callbacks import Callback, EarlyStopping

# Redirect console output to a file
log_file = '/app/data/code/console_output_small.log'
sys.stdout = open(log_file, 'w')

# Define a custom callback to capture accuracy at each epoch
class AccuracyLogger(Callback):
    def __init__(self, log_file):
        super().__init__()
        self.log_file = log_file
        # Open the log file in write mode
        with open(self.log_file, 'w') as f:
            f.write('Epoch,Accuracy,Validation Accuracy\n')

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        accuracy = logs.get('accuracy')
        val_accuracy = logs.get('val_accuracy')

        # Write the epoch, accuracy, and validation accuracy to the log file
        with open(self.log_file, 'a') as f:
            f.write(f"{epoch+1},{accuracy},{val_accuracy}\n")

# Define the dataset directory and number of classes
dataset_directory = '/app/data/Datasets/Trashnet-resized'   # 'Datasets/drinking_waste/Images_of_Waste/rawimgs/' 
num_classes = 6  # Set the number of classes

# log file for accuracy
log_file = '/app/data/code/accuracy_log_auto.csv'

# Create the hypermodel
hypermodel = WasteClassificationHyperModel(dataset_directory, num_classes)

# Define the tuner
tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='/app/data/Projtuning_results',
    project_name='waste_classification'
)

# Define the search space and perform tuning
tuner.search(
    create_data_generator(tuner.oracle.hyperparameters, dataset_directory),
    epochs=10,
    validation_data=create_data_generator(tuner.oracle.hyperparameters, dataset_directory),
    callbacks=[AccuracyLogger(log_file), EarlyStopping(patience=2)]
)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Print the summary of the best model
best_model.summary()

# Close the log file to ensure all output is written
sys.stdout.close()


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ MobileNetV3Small (Functional)   │ (None, 7, 7, 576)      │       939,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 3, 3, 576)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3, 3, 224)      │       129,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 3, 3, 224)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2016)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │        12,102 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,080,470 (4.12 MB)

 Trainable params: 141,350 (552.15 KB)

 Non-trainable params: 939,120 (3.58 MB)

In [1]:
# test for volume being loaded into docker image
import os

directory = '/app/data/' #Datasets/drinking_waste/Images_of_Waste'
if os.path.exists(directory):
    print("Directory exists")
    print(os.listdir(directory))
else:
    print("Directory does not exist")

Directory exists
['Datasets', 'code', 'README.md', 'accuracy_log1.csv', 'small_aug_plus_classification.ipynb', '.git']
